In [ ]:
import tqdm, re, pandas as pd, random, pprint
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
import datetime as dt
import pytz

In [ ]:
def get_memory():
    status= open('/proc/self/status').read()
    memuse= int(re.findall('VmRSS:\s*(\d+)\skB', status)[0]) / 1024 ** 2
    print('Resident memory: %.2f GB' % memuse)

In [ ]:
get_memory()

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

## Regular Users

### Tweet Level

In [ ]:
RegB= loadPickle('Regular_Before_py37.pkl')
RegA= loadPickle('Regular_After_py37.pkl')
regularBefore= RegB[RegB['user.id_str'].isin(RegA['user.id_str'])==True].copy()
regularAfter= RegA[RegA['user.id_str'].isin(RegB['user.id_str'])==True].copy()
regularBefore['typeOfUser']= 'restoredBefore'
regularAfter['typeOfUser']= 'restoredAfter'
regularBefore= regularBefore[['user.id_str', 'created_at']].copy()
regularAfter= regularAfter[['user.id_str', 'created_at']].copy()
RegA, RegB= None, None

In [ ]:
regular= pd.concat([regularBefore, regularAfter], ignore_index= True)
regularAfter, regularBefore= None, None

In [ ]:
regular['created_at']= pd.to_datetime(regular['created_at']).dt.tz_convert('Asia/Kolkata')
group_regular= regular.groupby('user.id_str')
fig= plt.figure()
fig.set_size_inches(12, 6)
ax= fig.add_subplot(1, 1, 1)
plt.xlabel('Date')
plt.ylabel('Number of Tweets')

for user_id, group in group_regular:
    
    user_dict= dict(group['created_at'].dt.date.value_counts())
    T= {}
    for j in user_dict:
        if user_dict[j]<300:
            T[j]= user_dict[j]
    
    x_0, y_0= [], []
    for i in {k: v for k, v in sorted(T.items(), key=lambda item: item[0])}:
        x_0.append(i)
        y_0.append(user_dict[i])
    
    plt.plot(x_0, y_0, c= ("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])))

plt.show()

In [ ]:
def nearestDates(items, pivot):
    D= [10**7, 10**7]
    dates= [None, None]
    for i in items:
        x= pivot-i
        # print(i, x.days)
        if x.days>0:
            if D[0]>x.days:
                dates[0]= i 
        elif x.days<0:
            if D[1]>abs(x.days):
                dates[1]= i
        else:
            return [pivot, pivot]
    return dates

In [ ]:
globalArr= {'before': [], 'after': []}
for user_id, group in group_regular:
    df= group # for each user tweets
    temp= df['created_at'].dt.tz_convert('Asia/Kolkata')
    piv= dt.datetime(2019, 6, 28, tzinfo= pytz.timezone('Asia/Kolkata'))
    div= nearestDates(temp, piv)
    globalArr['before'].append(div[0])
    globalArr['after'].append(div[1])
globalArr= pd.DataFrame.from_dict(globalArr)

In [ ]:
globalArr['Y']= 10

In [ ]:
globalArr.plot(kind= 'scatter', x= ['before', 'after'], y= ['Y', 'Y'], figsize=(10,5), grid=True, c= ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(894)]*2)

In [ ]:
temp= temp_cdf.get_lines()[0].get_data()
x_temp, y_temp= list(temp[0]), list(temp[1])
temp= temp_cdf.get_lines()[1].get_data()
x_temp_1, y_temp_1= list(temp[0]), list(temp[1])
plt.plot(x_temp_1, y_temp_1, label= "Regular Before", c= "#6B7280")
plt.plot(x_temp, y_temp, label= "Regular After", c= "#374151")
plt.ylim([0, 1])
plt.xscale('log', base= 10)
plt.xlabel('Favourite Count')
plt.ylabel('Proportion')
plt.legend()
plt.show()

## Restored Users 

### Retweet and Favourite Count - Tweet Level

In [ ]:
ResB= loadPickle('Restored_Before_py37.pkl')
ResA= loadPickle('Restored_After_py37.pkl')
restoredBefore= ResB[ResB['user.id_str'].isin(ResA['user.id_str'])==True].copy()
restoredAfter= ResA[ResA['user.id_str'].isin(ResB['user.id_str'])==True].copy()
restoredBefore['typeOfUser']= 'restoredBefore'
restoredAfter['typeOfUser']= 'restoredAfter'
restoredBefore= restoredBefore[['user.id_str', 'created_at']].copy()
restoredAfter= restoredAfter[['user.id_str', 'created_at']].copy()
ResA, ResB= None, None

In [ ]:
restored= pd.concat([restoredBefore, restoredAfter], ignore_index= True)
restoredBefore, restoredAfter= None, None

In [ ]:
restored['created_at']= pd.to_datetime(restored['created_at']).dt.tz_convert('Asia/Kolkata')
group_restored= restored.groupby('user.id_str')
fig= plt.figure()
fig.set_size_inches(12, 6)
ax= fig.add_subplot(1, 1, 1)
plt.xlabel('Date')
plt.ylabel('Number of Tweets')

for user_id, group in group_restored:
    
    user_dict= dict(group['created_at'].dt.date.value_counts())
    T= {}
    for j in user_dict:
        if user_dict[j]<300:
            T[j]= user_dict[j]
    
    x_0, y_0= [], []
    for i in {k: v for k, v in sorted(T.items(), key=lambda item: item[0])}:
        x_0.append(i)
        y_0.append(user_dict[i])
    
    plt.plot(x_0, y_0, c= ("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])))

plt.show()

In [ ]:
temp= temp_cdf.get_lines()[0].get_data()
x_temp, y_temp= list(temp[0]), list(temp[1])
temp= temp_cdf.get_lines()[1].get_data()
x_temp_1, y_temp_1= list(temp[0]), list(temp[1])
plt.plot(x_temp_1, y_temp_1, label= "Restored Before", c= "#6B7280")
plt.plot(x_temp, y_temp, label= "Restored After", c= "#374151")
plt.ylim([0, 1])
plt.xscale('log', base= 10)
plt.xlabel('Favourite Count')
plt.ylabel('Proportion')
plt.legend()
plt.show()